# Data Reshaping

Using some  basic and intermediate Pandas techniques, here's how I turned a file with 80 columns and 2 million rows into a usable excel file for a collegue to perform analysis on.

I started by importing Pandas, reading in the CSV file, and turning it into a dataframe called df.

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("/Users/jackfeen/Downloads/COVID-19_Vaccinations_in_the_United_States_County_20241016.csv")

/var/folders/7s/p_dkbml15ts8jfqmg89nydhc0000gn/T/ipykernel_45906/3897009208.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/jackfeen/Downloads/COVID-19_Vaccinations_in_the_United_States_County_20241016.csv")


In [83]:
pd.set_option('display.max_columns', None)
df = df.sort_values(by="Date", ascending=True)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Series_Complete_5to17,Series_Complete_5to17Pop_Pct,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Booster_Doses,Booster_Doses_Vax_Pct,Booster_Doses_5Plus,Booster_Doses_5Plus_Vax_Pct,Booster_Doses_12Plus,Booster_Doses_12Plus_Vax_Pct,Booster_Doses_18Plus,Booster_Doses_18Plus_Vax_Pct,Booster_Doses_50Plus,Booster_Doses_50Plus_Vax_Pct,Booster_Doses_65Plus,Booster_Doses_65Plus_Vax_Pct,Second_Booster_50Plus,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus,Second_Booster_65Plus_Vax_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_5PlusPop_Pct_SVI,Series_Complete_5to17Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_5to17Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Booster_Doses_Vax_Pct_SVI,Booster_Doses_12PlusVax_Pct_SVI,Booster_Doses_18PlusVax_Pct_SVI,Booster_Doses_65PlusVax_Pct_SVI,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop,Bivalent_Booster_5Plus,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct
1898994,01/01/2021,20173,53,Sedgwick County,KS,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,516042.0,NaN,NaN,428695.0,384853.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1899536,01/01/2021,08077,53,Mesa County,CO,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154210.0,NaN,NaN,132774.0,121401.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1899537,01/01/2021,19131,53,Mitchell County,IA,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10586.0,NaN,NaN,8975.0,8077.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1899538,01/01/2021,37003,53,Alexander County,NC,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37497.0,NaN,NaN,32781.0,30040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1899539,01/01/2021,55037,53,Florence County,WI,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4295.0,NaN,NaN,3903.0,3682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df.shape

(1962781, 80)

This code highlights columns with null values greater than 2 percent of the total records. If these columns were going to be of interest to him, I would need to decide on a strategy for imputation prior to grouping.

In [74]:
null_counts = df.isnull().sum()
null_percent = ((null_counts/1962781)*100).round(2)
sig_null_percent = null_percent[null_percent > 2]
if not sig_null_percent.empty:
    print(sig_null_percent)

Administered_Dose1_Recip                    2.97
Administered_Dose1_Recip_5Plus             61.93
Administered_Dose1_Recip_5PlusPop_Pct      61.69
Administered_Dose1_Recip_12Plus             5.99
Administered_Dose1_Recip_12PlusPop_Pct      2.48
Administered_Dose1_Recip_18Plus             5.06
Administered_Dose1_Recip_65Plus             5.06
Series_Complete_5Plus                      60.47
Series_Complete_5PlusPop_Pct               60.91
Series_Complete_5to17                      78.03
Series_Complete_5to17Pop_Pct               78.45
Booster_Doses                              61.53
Booster_Doses_Vax_Pct                      61.53
Booster_Doses_5Plus                        92.51
Booster_Doses_5Plus_Vax_Pct                92.51
Booster_Doses_12Plus                       68.70
Booster_Doses_12Plus_Vax_Pct               68.70
Booster_Doses_18Plus                       61.53
Booster_Doses_18Plus_Vax_Pct               61.53
Booster_Doses_50Plus                       61.53
Booster_Doses_50Plus

Next, I group the data by Metro Status, State, and Date, taking average for percent based columns and the sum for other numeric columns

Its worth noting that the 'mean' function for calculating percentages will return the average percent in a state by county, not the average for the state itself.

In [60]:
group_df = df.groupby(['Metro_status', 'Recip_State', 'Date'])

agg_funcs = {}

for col in df.select_dtypes(include='number').columns:
    if col.endswith('Pct') or col.endswith('pct'):
        agg_funcs[col] = 'mean'
    else:
        agg_funcs[col] = 'sum'

summed_df = group_df.agg(agg_funcs).reset_index()

In [61]:
summed_df.shape

(59800, 77)

In [63]:
summed_df.head()

,Metro_status,Recip_State,Date,MMWR_week,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Series_Complete_5to17,Series_Complete_5to17Pop_Pct,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Booster_Doses,Booster_Doses_Vax_Pct,Booster_Doses_5Plus,Booster_Doses_5Plus_Vax_Pct,Booster_Doses_12Plus,Booster_Doses_12Plus_Vax_Pct,Booster_Doses_18Plus,Booster_Doses_18Plus_Vax_Pct,Booster_Doses_50Plus,Booster_Doses_50Plus_Vax_Pct,Booster_Doses_65Plus,Booster_Doses_65Plus_Vax_Pct,Second_Booster_50Plus,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus,Second_Booster_65Plus_Vax_Pct,Series_Complete_Pop_Pct_SVI,Series_Complete_5PlusPop_Pct_SVI,Series_Complete_5to17Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_5to17Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Booster_Doses_Vax_Pct_SVI,Booster_Doses_12PlusVax_Pct_SVI,Booster_Doses_18PlusVax_Pct_SVI,Booster_Doses_65PlusVax_Pct_SVI,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop,Bivalent_Booster_5Plus,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct
0,Metro,AK,01/01/2021,159,0.0,0.0,0.000000,0.0,NaN,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493166.0,0.0,0.0,410914.0,372379.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1,Metro,AK,01/01/2022,156,96.7,306732.0,57.866667,306690.0,62.266667,296517.0,67.366667,276169.0,69.233333,53145.0,85.1,263181.0,49.666667,263173.0,53.433333,0.0,NaN,257522.0,58.466667,239929.0,60.1,48195.0,78.333333,87285.0,31.433333,0.0,NaN,0.0,NaN,86471.0,34.133333,55524.0,49.2,29817.0,60.2,0.0,NaN,0.0,NaN,22.0,22.0,0.0,23.0,23.0,23.0,10.0,10.0,0.0,11.0,11.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493166.0,458673.0,0.0,410914.0,372379.0,58340.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2,Metro,AK,01/02/2021,159,0.0,0.0,0.000000,0.0,NaN,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493166.0,0.0,0.0,410914.0,372379.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
3,Metro,AK,01/02/2022,3,96.7,306732.0,57.866667,306690.0,62.266667,296517.0,67.366667,276169.0,69.233333,53145.0,85.1,263181.0,49.666667,263173.0,53.433333,0.0,NaN,257522.0,58.466667,239929.0,60.1,48195.0,78.333333,87285.0,31.433333,0.0,NaN,0.0,NaN,86471.0,34.133333,55524.0,49.2,29817.0,60.2,0.0,NaN,0.0,NaN,22.0,22.0,0.0,23.0,23.0,23.0,10.0,10.0,0.0,11.0,11.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493166.0,458673.0,0.0,410914.0,372379.0,58340.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
4,Metro,AK,01/03/2021,3,0.0,0.0,0.000000,0.0,NaN,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0

In [64]:
summed_df.to_excel('summed_data_with_avg.xlsx', index=False)

In [65]:
print('done')

done


Using these techniques, I was able to reduce the csv file from ~2000000x80 to 59800x77! This will make the process of performing further analysis in excel signifcantly easier.